### Delmål 1: Data Exploration & Preprocessing (Uge 1)
**Status:** ⬜ Ikke startet

**Opgaver:**
- [ ] Download MovieLens dataset
- [ ] Exploratory Data Analysis (EDA) i Jupyter notebook
- [ ] Forstå data struktur: users, movies, ratings, timestamps
- [ ] Håndter missing values og outliers
- [ ] Train/validation/test split (70/15/15)
- [ ] Baseline statistik (gennemsnitlig rating, populære film)

**Færdig når:** EDA notebook er komplet med visualiseringer og data er splittet korrekt.


In [ ]:
# Packages 
import pandas as pd 

In [ ]:
# Load the data 
print("test")

In [ ]:
# Split the data into Train, val and test 

